In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

In [2]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [3]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_fake["class"] = 0
df_true["class"] = 1

In [6]:
print(df_fake.shape)
print(df_true.shape)

(23481, 5)
(21417, 5)


In [7]:
# Selecting the last 10 rows from the fake news DataFrame for manual testing
df_fake_manual_testing = df_fake.tail(10)

# Looping through the last 10 row indices of the fake news DataFrame
# and removing them from the DataFrame to prevent data leakage
for i in range(23480, 23470, -1):  # Indices 23480 to 23471
    df_fake.drop([i], axis=0, inplace=True)

# Selecting the last 10 rows from the true news DataFrame for manual testing
df_true_manual_testing = df_true.tail(10)

# Looping through the last 10 row indices of the true news DataFrame
# and removing them from the DataFrame to prevent data leakage
for i in range(21416, 21406, -1):  # Indices 21416 to 21407
    df_true.drop([i], axis=0, inplace=True)


In [8]:
print(df_fake.shape) 
print(df_true.shape)

(23471, 5)
(21407, 5)


In [9]:
# Assigning the class label '0' to all rows in the fake news manual testing DataFrame
df_fake_manual_testing["class"] = 0

# Assigning the class label '1' to all rows in the true news manual testing DataFrame
df_true_manual_testing["class"] = 1

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11028\860779283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11028\860779283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [10]:
df_fake_manual_testing

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [11]:
df_true_manual_testing

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [12]:
# Combining the fake and true news manual testing DataFrames into a single DataFrame
# by concatenating them along the rows (axis=0)
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)

In [13]:
df_manual_testing.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [ ]:
df_manual_testing.to_csv("manual_testing.csv")

In [9]:
# Merging the fake news and true news DataFrames into a single DataFrame
# by concatenating them along the rows (axis=0)
df_merge = pd.concat([df_fake, df_true], axis=0)

# Displaying the first 10 rows of the merged DataFrame
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [10]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

# Removing columns which are not required

In [11]:
# Dropping the "title," "subject," and "date" columns from the merged DataFrame
# to focus only on the relevant columns for further analysis
df = df_merge.drop(["title", "subject", "date"], axis=1)

In [12]:
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [13]:
df.shape

(44878, 2)

In [14]:
# Checking for any missing (null) values in each column of the DataFrame
# and displaying the count of null values for each column
df.isnull().sum()

text     0
class    0
dtype: int64

#  Random Shuffling the dataframe

In [15]:
# Shuffling the rows of the DataFrame randomly by sampling all rows with a fraction of 1 (100% of rows)
# This ensures the data is mixed thoroughly to avoid any order bias
df = df.sample(frac=1)

In [16]:
df.head(10)

,text,class
13641,BRUSSELS (Reuters) - North Korea s latest miss...,1
3298,(Reuters) - Former FBI Director James Comey sa...,1
11361,Enough is enough. Americans need to start maki...,0
20423,"VARADERO, Cuba (Reuters) - From Cuba to Antigu...",1
316,"Hurricane Irma, one of the most powerful Atlan...",0
3737,WASHINGTON (Reuters) - President Donald Trump ...,1
14206,The optics are horrible when there s a terror ...,0
3771,One of the things that Trump wants to do over ...,0
12750,,0
5039,As Donald Trump continues to be showered with ...,0


In [17]:
df.head()

,text,class
13641,BRUSSELS (Reuters) - North Korea s latest miss...,1
3298,(Reuters) - Former FBI Director James Comey sa...,1
11361,Enough is enough. Americans need to start maki...,0
20423,"VARADERO, Cuba (Reuters) - From Cuba to Antigu...",1
316,"Hurricane Irma, one of the most powerful Atlan...",0


# Creating a function to process the texts

In [18]:
# Function to preprocess text by cleaning and standardizing it
def wordopt(text):
    # Convert all characters in the text to lowercase
    text = text.lower()
    
    # Remove text within square brackets (e.g., [example])
    text = re.sub('\[.*?\]', '', text)
    
    # Remove all non-alphanumeric characters (e.g., symbols, punctuation) and replace with a space
    text = re.sub("\\W", " ", text)
    
    # Remove URLs starting with http://, https://, or www.
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags (e.g., <p>, <a>)
    text = re.sub('<.*?>+', '', text)
    
    # Remove punctuation by escaping special characters
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # Remove newline characters
    text = re.sub('\n', '', text)
    
    # Remove words containing digits (e.g., "abc123" or "123abc")
    text = re.sub('\w*\d\w*', '', text)
    
    # Return the cleaned text
    return text


In [19]:
# Applying the `wordopt` function to the "text" column of the DataFrame
# to clean and preprocess the text in each row
df["text"] = df["text"].apply(wordopt)

In [20]:
df.head()

,text,class
13641,brussels reuters north korea s latest miss...,1
3298,reuters former fbi director james comey sa...,1
11361,enough is enough americans need to start maki...,0
20423,varadero cuba reuters from cuba to antigu...,1
316,hurricane irma one of the most powerful atlan...,0


In [21]:
# Import necessary modules from the NLTK library for text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer and stop words set
lemmatizer = WordNetLemmatizer()  # For lemmatization of words
stop_words = set(stopwords.words('english'))  # Set of common stopwords to remove

# Define a function to preprocess the text
def preprocess_text(text):
    # Tokenize the text by converting it to lowercase and splitting it into individual words (tokens)
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords and non-alphanumeric words, then lemmatize each word
    words = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    
    # Return the preprocessed words as a single string
    return ' '.join(words)

# Apply the preprocessing function to each row in the "text" column of the DataFrame
df['text'] = df['text'].apply(preprocess_text)

# Display the first few rows of the DataFrame after preprocessing
df.head()

,text,class
13641,brussels reuters north korea latest missile la...,1
3298,reuters former fbi director james comey said t...,1
11361,enough enough american need start making busin...,0
20423,varadero cuba reuters cuba antigua caribbean i...,1
316,hurricane irma one powerful atlantic storm eve...,0


In [22]:
# Resetting the index and dropping the old index
df = df.reset_index(drop=True)

# Displaying the updated DataFrame
df.head()


,text,class
0,brussels reuters north korea latest missile la...,1
1,reuters former fbi director james comey said t...,1
2,enough enough american need start making busin...,0
3,varadero cuba reuters cuba antigua caribbean i...,1
4,hurricane irma one powerful atlantic storm eve...,0


# Defining dependent and independent variables

In [23]:
# Assigning the "text" column to the variable `x` (features) 
# and the "class" column to the variable `y` (target labels)
x = df["text"]
y = df["class"]

# Splitting Training and Testing¶

In [24]:
# Splitting the dataset into training and testing sets
# 25% of the data will be used for testing, and the remaining 75% will be used for training
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

# Convert text to vectors

In [25]:
# Importing the TfidfVectorizer from scikit-learn to convert text data into TF-IDF features
from sklearn.feature_extraction.text import TfidfVectorizer

# Initializing the TfidfVectorizer to convert text into a matrix of TF-IDF features
vectorization = TfidfVectorizer()

# Fitting the vectorizer to the training data and transforming it into TF-IDF features
xv_train = vectorization.fit_transform(x_train)

# Transforming the test data into TF-IDF features using the same fitted vectorizer
xv_test = vectorization.transform(x_test)


In [26]:
print(xv_train)

  (0, 22160)	0.36131839876388594
  (0, 14671)	0.21332341663330526
  (0, 72946)	0.42120393635273873
  (0, 68342)	0.1603566665663671
  (0, 76728)	0.1545185046890861
  (0, 23808)	0.364570926629137
  (0, 72844)	0.15359470963775967
  (0, 54883)	0.28090490031949283
  (0, 24545)	0.22615363084057197
  (0, 75215)	0.1455076276766086
  (0, 38481)	0.4287805997296093
  (0, 39420)	0.22577247300462758
  (0, 44910)	0.2332045126212785
  (1, 4747)	0.02241361438039365
  (1, 17529)	0.02241361438039365
  (1, 43797)	0.03791937764458616
  (1, 71830)	0.02135788933244772
  (1, 69824)	0.018854270128319507
  (1, 20868)	0.019918068611716935
  (1, 37013)	0.017629006434354683
  (1, 15211)	0.019799622428752283
  (1, 73948)	0.046575391860350264
  (1, 33252)	0.09024644223313111
  (1, 67570)	0.054167721731209546
  (1, 15512)	0.054167721731209546
  :	:
  (33657, 45881)	0.02508257371813302
  (33657, 32321)	0.0631948879645863
  (33657, 34227)	0.03776153230039857
  (33657, 61110)	0.03178141423249579
  (33657, 85025)	0.0799

In [27]:
print(xv_test)

  (0, 85728)	0.02825110559704951
  (0, 85021)	0.06656638321139026
  (0, 85018)	0.037118803361132255
  (0, 84988)	0.04281361475192306
  (0, 84026)	0.02282357610090183
  (0, 83996)	0.41836033690241115
  (0, 83740)	0.039800419902260235
  (0, 83433)	0.01910318137010063
  (0, 82822)	0.04164500125752831
  (0, 82726)	0.02597128016296873
  (0, 82720)	0.03135727803585764
  (0, 82719)	0.021759222267693154
  (0, 82684)	0.04464447242565208
  (0, 80580)	0.035815289435070494
  (0, 78732)	0.11863998156454997
  (0, 76913)	0.0347738913852599
  (0, 75685)	0.03547109153672136
  (0, 75237)	0.0257040264786842
  (0, 75215)	0.01899011091134655
  (0, 74391)	0.0400933593188181
  (0, 73713)	0.03480425558609371
  (0, 73672)	0.03991892558809344
  (0, 73670)	0.07903400593279734
  (0, 73212)	0.021374577036708097
  (0, 72824)	0.0403206372599537
  :	:
  (11219, 4098)	0.015049976573917757
  (11219, 4073)	0.020902409410894596
  (11219, 3902)	0.018005966516105122
  (11219, 3745)	0.07336756165264727
  (11219, 3193)	0.027

In [31]:
# Initialize the Logistic Regression model with a fixed random state for reproducibility
model = LogisticRegression(random_state=10)

# Train the model using the training data (TF-IDF features and corresponding labels)
model.fit(xv_train, y_train)

# Make predictions on the test data (using the trained model)
y_pred = model.predict(xv_test)

# Evaluate the model's performance using different metrics:
# Accuracy: The proportion of correct predictions
accuracy = accuracy_score(y_test, y_pred)

# Recall: The proportion of actual positives correctly identified by the model
recall = recall_score(y_test, y_pred)

# Precision: The proportion of predicted positives that are actually positive
precision = precision_score(y_test, y_pred)

# F1 Score: The harmonic mean of precision and recall, useful for imbalanced classes
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)


Accuracy: 0.9849376114081997
Recall: 0.9868766404199475
Precision: 0.9815401827335447
F1 Score: 0.984201177900346


In [29]:
# Initialize the Decision Tree Classifier model with a fixed random state for reproducibility
model = DecisionTreeClassifier(random_state=10)

# Train the model using the training data (TF-IDF features and corresponding labels)
model.fit(xv_train, y_train)

# Make predictions on the test data (using the trained model)
y_pred = model.predict(xv_test)

# Evaluate the model's performance using different metrics:

# Accuracy: The proportion of correct predictions
accuracy = accuracy_score(y_test, y_pred)

# Recall: The proportion of actual positives correctly identified by the model
recall = recall_score(y_test, y_pred)

# Precision: The proportion of predicted positives that are actually positive
precision = precision_score(y_test, y_pred)

# F1 Score: The harmonic mean of precision and recall, useful for imbalanced classes
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)

Accuracy: 0.9952762923351158
Recall: 0.9962504686914135
Precision: 0.9938283149429586
F1 Score: 0.995037917797959


In [30]:
# Initialize the Random Forest Classifier model with a fixed random state for reproducibility
model = RandomForestClassifier(random_state=10)

# Train the model using the training data (TF-IDF features and corresponding labels)
model.fit(xv_train, y_train)

# Make predictions on the test data (using the trained model)
y_pred = model.predict(xv_test)

# Evaluate the model's performance using different metrics:

# Accuracy: The proportion of correct predictions
accuracy = accuracy_score(y_test, y_pred)

# Recall: The proportion of actual positives correctly identified by the model
recall = recall_score(y_test, y_pred)

# Precision: The proportion of predicted positives that are actually positive
precision = precision_score(y_test, y_pred)

# F1 Score: The harmonic mean of precision and recall, useful for imbalanced classes
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)

Accuracy: 0.9900178253119429
Recall: 0.9911886014248219
Precision: 0.9878550074738416
F1 Score: 0.9895189968182669


In [32]:
import joblib

# Save the trained model
joblib.dump(model, 'logistic_regression_model.pkl')
print("Model saved successfully!")

# Save the vectorizer
joblib.dump(vectorization, 'tfidf_vectorizer.pkl')
print("Vectorizer saved successfully!")


Model saved successfully!
Vectorizer saved successfully!


In [33]:
# Load the saved model and vectorizer
loaded_model = joblib.load('logistic_regression_model.pkl')
loaded_vectorizer = joblib.load('tfidf_vectorizer.pkl')

# Input a new article
new_article = input("Enter a new article for classification: ")

# Vectorize the new article using the loaded TfidfVectorizer
new_article_vectorized = loaded_vectorizer.transform([new_article])

# Predict the class of the new article
new_prediction = loaded_model.predict(new_article_vectorized)

# Display the prediction
print("Predicted class for the new article:", new_prediction[0])


Enter a new article for classification: Vic Bishop Waking TimesOur reality is carefully constructed by powerful corporate, political and special interest sources in order to covertly sway public opinion. Blatant lies are often televised regarding terrorism, food, war, health, etc. They are fashioned to sway public opinion and condition viewers to accept what have become destructive societal norms.The practice of manipulating and controlling public opinion with distorted media messages has become so common that there is a whole industry formed around this. The entire role of this brainwashing industry is to figure out how to spin information to journalists, similar to the lobbying of government. It is never really clear just how much truth the journalists receive because the news industry has become complacent. The messages that it presents are shaped by corporate powers who often spend millions on advertising with the six conglomerates that own 90% of the media:General Electric (GE), N

In [35]:
# Input a new article
new_article = input("Enter a new article for classification: ")

# Vectorize the new article using the loaded TfidfVectorizer
new_article_vectorized = loaded_vectorizer.transform([new_article])

# Predict the class of the new article
new_prediction = loaded_model.predict(new_article_vectorized)

# Display the prediction
print("Predicted class for the new article:", new_prediction[0])

Enter a new article for classification: MOSCOW (Reuters) - Vatican Secretary of State Cardinal Pietro Parolin said on Tuesday that there was  positive momentum  behind the idea of Pope Francis visiting Russia, but suggested there was more work to be done if it were to happen.  Parolin, speaking at a joint news conference in Moscow alongside Russian Foreign Minister Sergei Lavrov, did not give any date for such a possible visit. The Eastern and Western branches of Christianity split apart in 1054. The pope, leader of the world s 1.2 billion Catholics, is seeking to improve ties, and last year in Cuba held what was the first ever meeting between a Roman Catholic pope and a Russian Orthodox patriarch.  Parolin said he had also used his talks in the Russian capital to also raise certain difficulties faced by the Catholic Church in Russia. He said that Moscow and the Vatican disagreed about the plight of Christians in certain parts of the world. He did not elaborate. Parolin, who is due lat